In [ ]:
import pickle
import numpy as np
import pandas as pd

DATA_TYPE = 'titanic'
if DATA_TYPE == 'weather':
    data_dir = '../data/'
    filename = 'city_temperature.csv'
    df = pd.read_csv(data_dir + filename)
    df['Region'].value_counts()/len(df)
    yes_label = 'North America'
    df['Label'] = df.apply(lambda x: 1 if x['Region'] == yes_label else 0, axis=1)
    df.replace(-99.0, np.nan, inplace=True)
    df.dropna(subset=['AvgTemperature'], inplace=True)
    df['State'].fillna('NA', inplace=True)
    df = df.groupby(['Year', 'Region', 'Country', 'State', 'City'])['AvgTemperature'].apply(list)
    df = df.reset_index()
    df['Label'] = df.apply(lambda x: 1 if x['Region'] == yes_label else 0, axis=1)
    df['len_ts'] = df['AvgTemperature'].apply(lambda x: len(x))

    def day_ts_to_month_ts(ts):
        months = [0, 31, 31+28, 31+28+31, 31+28+31+30, 31+28+31+30+31, 31+28+31+30+31+30, 31+28+31+30+31+30+31, 31+28+31+30+31+30+31+31, 31+28+31+30+31+30+31+31+30, 31+28+31+30+31+30+31+31+30+31, 31+28+31+30+31+30+31+31+30+31+30, 31+28+31+30+31+30+31+31+30+31+30+31]
        if len(ts) == 366: # leap year
            for i in range(2, 13):
                months[i] += 1
        elif len(ts) != 365:
            return None
        month_ts = []
        for i in range(1, 13):
            month_ts.append(int(round(np.mean(ts[months[i-1]:months[i]]), 0)))
        return month_ts

    df['month_ts'] = df['AvgTemperature'].apply(day_ts_to_month_ts)
    df['month_ts'] = df['AvgTemperature'].apply(day_ts_to_month_ts)
    df.dropna(inplace=True)
    df[['temp1', 'temp2', 'temp3', 'temp4', 'temp5', 'temp6', 'temp7', 'temp8', 'temp9', 'temp10', 'temp11', 'temp12']] = pd.DataFrame(df['month_ts'].tolist(), index=df.index)
    data_dict = df[['Year', 'Region', 'Country', 'State', 'City', 'month_ts', 'Label']].rename(columns={'month_ts': 'MonthlyAvgTemperature'}).to_dict(orient='index')

    with open('../data/weather.pkl', 'wb') as handle:
        pickle.dump(data_dict, handle)

elif DATA_TYPE == 'vitals':
    # TODO: manually specify units?
    data_dict = pickle.load(open('../data/vitals_raw.pkl', 'rb'))
    d_items = pd.read_csv('/Users/kylecox/Documents/ws/mimic-llm/prompting/data/d_items.csv')
    d_items_dict = {d_items['itemid'][i]: d_items['label'][i] for i in range(len(d_items))}
    new_data_dict = {}
    for key in data_dict:
        new_data_dict[key] = {}
        for label, val in data_dict[key].items():
            if type(val) == list:
                new_label = d_items_dict[int(label)]
                new_list = [int(round(el)) for el in val]
                new_list = [el if el != 0 else 'NA' for el in new_list]
                new_data_dict[key][new_label] = new_list
            elif label == 'label':
                new_data_dict[key]['Label'] = val
            else:
                new_data_dict[key][label] = val
    with open('../data/vitals.pkl', 'wb') as handle:
        pickle.dump(new_data_dict, handle)

elif DATA_TYPE == 'titanic':
    df = pd.read_csv('../data/titanic.csv', index_col=0)
    label = 'Survived'
    feature_names = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare']
    df = df[feature_names + [label]]
    rename_cols = {label: 'Label', 'Pclass': 'Ticket Class', 'Age': 'Age (years)', 'SibSp': 'Number of Siblings and Spouses Aboard', 'Parch': 'Number of Parents and Children Aboard', 'Fare': 'Fare Price'}
    df.rename(columns=rename_cols, inplace=True)
    df.dropna(inplace=True)
    for col in df.columns:
        print(df[col].value_counts() / len(df))
        print()
    data_dict = {}
    for idx, row in df.iterrows():
        data_dict[idx] = {}
        for col, val in row.items():
            data_dict[idx][col] = val
    with open('../data/titanic.pkl', 'wb') as handle:
        pickle.dump(data_dict, handle)

In [5]:
import pandas as pd
import pickle
filename = 'data/data_frame/housing.csv'
int_cols = ['Housing Median Age', 'Total Rooms', 'Population', 'Number Households', 'Median Income ($)', 'Label']
df = pd.read_csv(filename)
df.index = range(len(df))
print(df.head())
data_dict = {}
for idx, row in df.iterrows():
    data_dict[idx] = {}
    for col, val in row.items():
        if col in int_cols:
            data_dict[idx][col] = int(val)
        else:
            data_dict[idx][col] = val
            

# save data_dict to pickle
filename = 'data/data_dict/housing.pkl'
with open(filename, 'wb') as handle:
    pickle.dump(data_dict, handle)


   Longitude  Latitude  Housing Median Age  Total Rooms  Total Bedrooms  \
0    -122.23     37.88                  41          880           129.0   
1    -122.22     37.86                  21         7099          1106.0   
2    -122.24     37.85                  52         1467           190.0   
3    -122.25     37.85                  52         1274           235.0   
4    -122.25     37.85                  52         1627           280.0   

   Population  Number Households  Median Income ($)  Label  
0         322                126              83252      1  
1        2401               1138              83014      1  
2         496                177              72574      1  
3         558                219              56431      1  
4         565                259              38462      1  
